# Multiple Linear Regression with numerous features and Pandas Resampling

### The objective behind this exercise is to predict weekly sales across several Walmart stores in the US. There are numerous features that go into this prediction, some of which are categorical and the others, numeric.  

We will also estimate sales in each quarter for all stores and address the following question: **which stores experienced the highest and lower Quarter-Quarter sales %increase and %decline in any given period**

In [67]:
#import relevant packages and files into the Jupyter Lab Environment

import numpy as np
import pandas as pd

consumer_features = pd.read_csv(r"C:\Users\LENOVO\Desktop\Data Science\PYTHON and DATAI\DATAI.HW1\features.csv")
stores = pd.read_csv(r'C:\Users\LENOVO\Desktop\Data Science\PYTHON and DATAI\DATAI.HW1\stores.csv')
train = pd.read_csv(r'C:\Users\LENOVO\Desktop\Data Science\PYTHON and DATAI\DATAI.HW1\train.csv')
test = pd.read_csv(r'C:\Users\LENOVO\Desktop\Data Science\PYTHON and DATAI\DATAI.HW1\test.csv')

In [2]:
consumer_features.head(n = 15)

,Store,Date,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday
0,1,2/5/2010,42.31,2.572,211.096358,8.106,False
1,1,2/12/2010,38.51,2.548,211.242170,8.106,True
2,1,2/19/2010,39.93,2.514,211.289143,8.106,False
3,1,2/26/2010,46.63,2.561,211.319643,8.106,False
4,1,3/5/2010,46.50,2.625,211.350143,8.106,False
5,1,3/12/2010,57.79,2.667,211.380643,8.106,False
6,1,3/19/2010,54.58,2.720,211.215635,8.106,False
7,1,3/26/2010,51.45,2.732,211.018042,8.106,False
8,1,4/2/2010,62.27,2.719,210.820450,7.808,False
9,1,4/9/2010,65.86,2.770,210.622857,7.808,False


The significance of this datasheet is to pinpoint the features that go into predicting whether there is a nationwide holiday in any given week throughout the year from 2010 to 2013. This categorical outcome will be used later as a feature itself to predict weekly sales in Walmart stores.

In [3]:
stores.head(n = 15)


,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875
5,6,A,202505
6,7,B,70713
7,8,A,155078
8,9,B,125833
9,10,B,126512


In [4]:
train.head(n = 15)

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2/5/2010,24924.50,False
1,1,1,2/12/2010,46039.49,True
2,1,1,2/19/2010,41595.55,False
3,1,1,2/26/2010,19403.54,False
4,1,1,3/5/2010,21827.90,False
5,1,1,3/12/2010,21043.39,False
6,1,1,3/19/2010,22136.64,False
7,1,1,3/26/2010,26229.21,False
8,1,1,4/2/2010,57258.43,False
9,1,1,4/9/2010,42960.91,False


In [5]:
test.head(n = 15)

,Store,Dept,Date,IsHoliday
0,1,1,11/2/2012,False
1,1,1,11/9/2012,False
2,1,1,11/16/2012,False
3,1,1,11/23/2012,True
4,1,1,11/30/2012,False
5,1,1,12/7/2012,False
6,1,1,12/14/2012,False
7,1,1,12/21/2012,False
8,1,1,12/28/2012,True
9,1,1,1/4/2013,False


## Data Wrangling and Features Engineering

In [6]:
#Merge the attributes in the Store dataset with the training set for prediction.

train = pd.merge(train,stores, how = 'inner')
train

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size
0,1,1,2/5/2010,24924.50,False,A,151315
1,1,1,2/12/2010,46039.49,True,A,151315
2,1,1,2/19/2010,41595.55,False,A,151315
3,1,1,2/26/2010,19403.54,False,A,151315
4,1,1,3/5/2010,21827.90,False,A,151315
...,...,...,...,...,...,...,...
421565,45,98,9/28/2012,508.37,False,B,118221
421566,45,98,10/5/2012,628.10,False,B,118221
421567,45,98,10/12/2012,1061.02,False,B,118221
421568,45,98,10/19/2012,760.01,False,B,118221


In [7]:
#check for missing values and datatypes

pd.DataFrame({'Missing values % in each column':((pd.isnull(train).sum()/len(train))*100)}).reset_index()



,index,Missing values % in each column
0,Store,0.0
1,Dept,0.0
2,Date,0.0
3,Weekly_Sales,0.0
4,IsHoliday,0.0
5,Type,0.0
6,Size,0.0


In [8]:
# We are not dealing with any missing values in this datasheet, so we can now proceed to check datatypes
train.dtypes

Store             int64
Dept              int64
Date             object
Weekly_Sales    float64
IsHoliday          bool
Type             object
Size              int64
dtype: object

In [9]:
#We need to convert the date column to its correct type. 

#Store and Department numbers must be treated as categorical.

In [10]:
train['Date'] = pd.to_datetime(train['Date'])

#We will make use of the get_dummies function to ensure that we make categorically representative data ready for use.

train = pd.merge(train, pd.get_dummies(train['Type'], prefix = 'type'), left_index = True, right_index = True)
train.drop(columns = ['Type'], inplace = True)

X = np.array(train[['Store','Dept','IsHoliday','Size', 'type_A', 'type_B', 'type_C']].values).reshape(-1,7)
y = np.array(train['Weekly_Sales'].values).reshape(-1,1)

train['Store'] = train['Store'].astype("category")
train['Dept'] = train['Dept'].astype("category")

train

,Store,Dept,Date,Weekly_Sales,IsHoliday,Size,type_A,type_B,type_C
0,1,1,2010-02-05,24924.50,False,151315,1,0,0
1,1,1,2010-02-12,46039.49,True,151315,1,0,0
2,1,1,2010-02-19,41595.55,False,151315,1,0,0
3,1,1,2010-02-26,19403.54,False,151315,1,0,0
4,1,1,2010-03-05,21827.90,False,151315,1,0,0
...,...,...,...,...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False,118221,0,1,0
421566,45,98,2012-10-05,628.10,False,118221,0,1,0
421567,45,98,2012-10-12,1061.02,False,118221,0,1,0
421568,45,98,2012-10-19,760.01,False,118221,0,1,0


In [11]:
train.dtypes

Store                 category
Dept                  category
Date            datetime64[ns]
Weekly_Sales           float64
IsHoliday                 bool
Size                     int64
type_A                   uint8
type_B                   uint8
type_C                   uint8
dtype: object

We can now proceed to fit our model

In [12]:
#Training the model
from sklearn.model_selection import train_test_split

x_train, x_text, y_train,y_test = train_test_split(X,y,test_size = len(test)/len(train), random_state = 1)
#Fitting the linear regression model
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(x_train,y_train)
print("Regression Coefficients ", reg.coef_)
print("Intercept", reg.intercept_)


Regression Coefficients  [[-1.25813924e+02  1.10767723e+02  1.03271832e+03  1.00122997e-01
  -1.83176261e+03 -1.85748924e+03  3.68925184e+03]]
Intercept [1384.27709222]


#### We can now proceed with resampling the data to predict Quarterly Sales across stores and departments.


In [13]:
#Quarterly Sales prediction

quarter_train = train

quarter_train.drop(columns = ['IsHoliday', 'Size','type_A','type_B','type_C'], inplace = True)
quarter_train = quarter_train.set_index('Date')

quarter_train2 = quarter_train.groupby(['Store','Dept']).resample('Q').sum()
quarter_train2 = quarter_train2.reset_index()
quarter_train2 = quarter_train2.set_index('Date')

quarter_train2.rename(columns = {'Weekly_Sales':'Quarterly_Sales'}, inplace = True)
quarter_train2

,Store,Dept,Quarterly_Sales
Date,,,
2010-03-31,1,1,223200.22
2010-06-30,1,1,283951.61
2010-09-30,1,1,218407.09
2010-12-31,1,1,400789.81
2011-03-31,1,1,277732.56
...,...,...,...
2011-12-31,45,98,10665.02
2012-03-31,45,98,9462.34
2012-06-30,45,98,9449.91


In [66]:
#creating a column for quarterly sales to compare maximum % increases and declines in sales

quarter_train2['percent change (%)'] = (quarter_train2['Quarterly_Sales'].pct_change())*100

quarter_train2.head(n = 20)

,Store,Dept,Quarterly_Sales,percent change (%)
Date,,,,
2010-09-30,1,1,218407.09,NaN
2010-12-31,1,1,400789.81,83.505861
2011-03-31,1,1,277732.56,-30.703687
2011-06-30,1,1,296287.24,6.680772
2011-09-30,1,1,225674.59,-23.832498
2011-12-31,1,1,371856.41,64.775489
2012-03-31,1,1,319569.19,-14.061132
2012-06-30,1,1,280039.64,-12.369637
2012-09-30,1,1,225652.26,-19.421315


We see that some of the percentage changes happen to be negative and positive infinity. We will address this issue by first determining what percentage of the dataframe consists of these infeasible values. Depending on the percentage (high or low), we will develop a suitable strategy to resolve this issue.

In [15]:
(len(quarter_train2[quarter_train2['percent change (%)'] == np.inf])/len(quarter_train2))*100

0.6626362892367803

Since only less than 1% of values are infeasible, we can safely drop these rows from our dataframe and proceed for a more accurate analysis.

In [61]:
quarter_train2 = quarter_train2.replace(np.inf,np.nan)
quarter_train2 = quarter_train2.replace(-np.inf,np.nan)

quarter_train2.dropna(inplace = True)
quarter_train2.head(n = 20)

,Store,Dept,Quarterly_Sales,percent change (%)
Date,,,,
2010-09-30,1,1,218407.09,-23.082989
2010-12-31,1,1,400789.81,83.505861
2011-03-31,1,1,277732.56,-30.703687
2011-06-30,1,1,296287.24,6.680772
2011-09-30,1,1,225674.59,-23.832498
2011-12-31,1,1,371856.41,64.775489
2012-03-31,1,1,319569.19,-14.061132
2012-06-30,1,1,280039.64,-12.369637
2012-09-30,1,1,225652.26,-19.421315


We can now proceed to find the largest Q-Q sales decline and increase per store and department. Subsequently, we will extract 2 specific stores and departments that have the **highest magnitude** of %change in their sales from all the data collectively.

In [65]:
#creating a column for quarterly sales to compare maximum % increases and declines in sales

u = quarter_train2.groupby(['Store','Dept'])['percent change (%)'].max() #find the maximum %increase in Q-Q sales for each store and department
v = quarter_train2.groupby(['Store','Dept'])['percent change (%)'].min() #find the maximum %decline in Q-Q sales for each store and department

highest_sales_percentage_increase = u.max()
steepest_sales_percentage_decline = v.min()


#output
print("Store with largest sales % increase between Quarters:",quarter_train2.loc[quarter_train2['percent change (%)'] == highest_sales_percentage_increase, ['Store','Dept']])
print("Store with largest sales % decrease between Quarters:",quarter_train2.loc[quarter_train2['percent change (%)'] == steepest_sales_percentage_decline, ['Store','Dept']])

Store with largest sales % increase between Quarters:            Store Dept
Date                 
2010-03-31    37    1
Store with largest sales % decrease between Quarters:            Store Dept
Date                 
2010-03-31    15   38


## Conclusions

### Our analysis leads us to conclude the following:

Department 1 in Store 37 experienced the largest Quarter-to Quarter sales % increase of all the Walmart stores and Departments in our analysis.

Department 38 in Store 15 experienced the largest magnitude Quarter-to Quarter sales % decrease of all the Walmart stores and Departments in our analysis.
